In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import wget
from collections import namedtuple
Fallo = namedtuple('Fallo',['nombre','informacion'])

output_directory='./pdf_fallos/'

ModuleNotFoundError: ignored

In [0]:
def extraer_informacion(resultado=None):
    texto = resultado.find_elements_by_tag_name('li')
    link = resultado.find_element_by_class_name("download").get_attribute("href")
    informacion = {'link':link}
    for linea in texto:
        campo, info = tuple(t.strip() for t in linea.text.split(':',1))
        informacion[campo] = info
    return informacion

def extraer_dato_menu(nombre=""):
    element =  driver.find_element_by_name(nombre)
    value = [x.text for x in element.find_elements_by_tag_name("option")\
         if x.get_attribute('value') == element.get_attribute('value')]
    return value[0]

def extraer_dato(nombre=""):
    element = driver.find_element_by_name(nombre)
    return element.get_attribute('value')


In [0]:
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
driver.get("http://cij.gov.ar/sentencias.html")

In [0]:
fallos_totales = int(driver.find_elements_by_class_name("s1")[0].text)

tipo_fallo = extraer_dato_menu("tipofallo")
tribunal = extraer_dato_menu("tid")
jurisdiccion = extraer_dato_menu("jurisdiccion")
palabras_claves = extraer_dato("firmantes")
expediente =  extraer_dato("expediente")
caratula =  extraer_dato("caratula")
fecha_desde = extraer_dato("fecha_fallo_desde")
fecha_hasta = extraer_dato("fecha_fallo_hasta")
metadata_busqueda = {
    'fallos_totales':fallos_totales,
    'palabras_clave':palabras_claves,
    'tipo_fallo':tipo_fallo,
    'jurisdiccion':jurisdiccion,
    'tribunal':tribunal,
    'expediente':expediente,
    'caratula':caratula,
    'intervalo':[fecha_desde,fecha_hasta]
}
generador_nro_fallo = (i for i in range(1,fallos_totales+1))

In [0]:
files = []
n_paginas =  fallos_totales // 20 # muestra 20 resultados por página 
for i in range(n_paginas+1):
    driver.execute_script('irPaginaF({})'.format(i))
    resultados = driver.find_elements_by_class_name("result")
    assert len(resultados) > 0
    for resultado in resultados:
        nro_archivo = next(generador_nro_fallo)
        filename = 'fallo_{:04d}.pdf'.format(nro_archivo)
        informacion = extraer_informacion(resultado)
        wget.download(informacion['link'],out=output_directory+filename)
        fallo = Fallo(filename,informacion)
        files.append(fallo)
    print("\r[{0}] {1:d}/{2:d}".format('='*int(60*nro_archivo/fallos_totales), nro_archivo,fallos_totales),end=' ')
    print("\r",end='')  

In [0]:
import json
with open('metadata_busqueda.json','w') as f:
    json.dump(metadata_busqueda,f)
with open('info_fallos.json','w') as f:
    json.dump([x._asdict() for x in files],f)